### Tasks 

1. Alert on buses which have become delayed in the last 15 minutes
2. Visialize bus current location
3. Count of buses per operator which are actively moving in the last 15 minutes

### Needed data
1. Veichle id: MonitoredVehicleJourney -> VehicleRef
2. Veichle location: MonitoredVehicleJourney -> VehicleLocation
3. Event time: RecordedAtTime
4. Operator: MonitoredVehicleJourney -> OperatorRef
5. VehicleAtStop -> {true, false}, moving or not
6. Delayed status: MonitoredVehicleJourney -> OnwardCalls -> OnwardCall -> 0 -> ArrivalStatus
7. Veichle type: MonitoredVehicleJourney -> VehicleMode

#### Project Plan
Load data from API (at XXX seconds intervals)
Extract relevant fields into a dictionary
Push the dictionary to a kafka topic 
Consume the kafka topic with a spark job
Extract veiechles with delayed arrival status and print the alert
Extract veiehcle location data and feed the data into a map visualization tool
Extract the veiechles with the actively moveing status, group them by operator.




In [31]:
!pip install xmltodict
!pip install confluent_kafka

You should consider upgrading via the 'c:\users\ahsor\appdata\local\programs\python\python37-32\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\ahsor\appdata\local\programs\python\python37-32\python.exe -m pip install --upgrade pip' command.


In [32]:
import itertools
import requests
from xml.etree import ElementTree
import xmltodict
from datetime import datetime
from confluent_kafka import Producer
import sched, time
import json

In [33]:
headers = {
    'content-type': 'application/json'
}


def apiCall(url):
    r = requests.get(url, headers=headers)
    return r.json()
    
url = "https://api.entur.io/realtime/v1/rest/vm?maxSize=2"
apiResult = apiCall(url)

In [34]:
def acked(err, msg):
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
    else:
        print("Message produced: %s" % (str(msg)))

def produce_events(apiResult):
    conf = {'bootstrap.servers': "localhost:19092,localhost:29092"}
    producer = Producer(conf)
    producer.produce(topic="AwsomeTopicName", value=str(apiResult), callback=acked)
    print("polling")
    producer.poll(1)
    print("flushing")
    producer.flush()
    print("All done")

In [35]:
instances = apiResult['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity']

for instance in instances:
    
    output = {"RecordedAtTime": instance['RecordedAtTime'],
              #"VehicleMode": instance['MonitoredVehicleJourney']['VehicleMode'],
              "OperatorRef": instance['MonitoredVehicleJourney']['OperatorRef'],
              "VehicleLocation": instance['MonitoredVehicleJourney']['VehicleLocation'],
              "VehicleRef": instance['MonitoredVehicleJourney']['VehicleRef'],
              "VehicleMode": instance['MonitoredVehicleJourney']['OnwardCalls']
    }

    produce_events(json.dumps(output))

polling
Message produced: <cimpl.Message object at 0x071FED40>
flushing
All done
polling
Message produced: <cimpl.Message object at 0x071FED98>
flushing
All done


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=abab48c6-a312-48d3-9d85-4f43cbfe0e3c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>